1. Importing necessary Libraries
2. Creating S3 Bucket
3. Mapping train and Test Data in S3
4. Mapping the path of models in S3

In [1]:
import sagemaker
import boto3
import time
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session
from sagemaker import image_uris

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/10/25 19:00:14] INFO     Found credentials from IAM Role:                                   ]8;id=277331;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=317156;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# bucket_name = f"bankapplication-jasonjoelpinto-{int(time.time())}"
bucket_name = 'bankapplication-jasonjoelpinto-1746892141'
sess = sagemaker.Session()
my_region = sess.boto_session.region_name
# my_region = "eu-north-1" 
print(my_region)
print(bucket_name)

                    INFO     Found credentials from IAM Role:                                   ]8;id=305772;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=266227;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

eu-north-1
bankapplication-jasonjoelpinto-1746892141


In [3]:
s3 = boto3.resource('s3',region_name=my_region)
try:
    if my_region == 'eu-north-1':
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={"LocationConstraint": my_region}
        )
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

[05/10/25 19:00:15] INFO     Found credentials from IAM Role:                                   ]8;id=909510;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=372759;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [4]:
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankapplication-jasonjoelpinto-1746892141/xgboost-as-a-built-in-algo/output


##### **Dataset Download**

In [5]:
import  pandas as pd
import urllib
try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv')
except Exception as e:
    print('Data Load error: ',e)


try:
    model_data = pd.read_csv('./bank_clean.csv', index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv
Success: Data loaded into dataframe.


In [6]:
model_data.head(10)

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
5,45,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
6,59,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
7,41,1,999,0,1,0,0,1,0,0,...,0,1,0,0,0,0,1,0,1,0
8,24,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
9,25,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


#### **Train Test Split**

In [7]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [8]:
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

[05/10/25 19:00:16] INFO     Found credentials from IAM Role:                                   ]8;id=437048;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=742535;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [9]:
import os
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

                    INFO     Found credentials from IAM Role:                                   ]8;id=95630;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=866853;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

### **Building && Training Models Xgboost- Inbuilt Algorithm**

In [10]:
container = image_uris.retrieve(
    framework='xgboost',
    region='eu-north-1',
    version='1.3-1'
)

container

[05/10/25 19:00:17] INFO     Ignoring unnecessary instance type: None.                            ]8;id=533208;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=846800;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

'662702820516.dkr.ecr.eu-north-1.amazonaws.com/sagemaker-xgboost:1.3-1'

#### **Initializing the Hyperparameters**

In [11]:
hyperparameters = {
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample":"0.7",
    "objective":"binary:logistic",
    "num_round":50
    }  

#### **Constructing the SageMaker estimator that will call the Xgboost-container**

In [12]:
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [13]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=75997;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=221095;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=883080;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=480620;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-05-10-19-00-17-322                                             

2025-05-10 19:00:22 Starting - Starting the training job...
2025-05-10 19:00:36 Starting - Preparing the instances for training...
2025-05-10 19:01:09 Downloading - Downloading the training image...
2025-05-10 19:01:39 Training - Training image download completed. Training in progress...[2025-05-10 19:01:57.614 ip-10-0-88-4.eu-north-1.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-05-10 19:01:57.635 ip-10-0-88-4.eu-north-1.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-05-10:19:01:57:INFO] Imported framework sagemaker_xgboost_container.training
[2025-05-10:19:01:57:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2025-05-10:19:01:57:INFO] No GPUs detected (normal if no gpus installed)
[2025-05-10:19:01:57:INFO] Running XGBoost Sagemaker in algorithm mode
[2025-05-10:19:01:57:INFO] Determined delimiter of CSV input is ','
[2025-05-10:19:01:57:INFO] Determine

In [14]:
print(s3_input_train)

In [15]:
from sagemaker import image_uris
image_uris.retrieve(framework='xgboost', region='eu-north-1', version='1.0-1')

[05/10/25 19:02:34] INFO     Defaulting to only available Python version: py3                     ]8;id=848004;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=782648;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#610\610]8;;\

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=725420;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=902706;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#534\534]8;;\

'662702820516.dkr.ecr.eu-north-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3'

#### **Deploying the Machine Learning Model**

In [18]:
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m5.large')

[05/10/25 19:07:56] INFO     Creating model with name: sagemaker-xgboost-2025-05-10-19-07-56-176    ]8;id=289458;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=554619;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=297614;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=626193;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\
                             sagemaker-xgboost-2025-05-10-19-07-56-176                                             

[05/10/25 19:07:57] INFO     Creating endpoint with name sagemaker-xgboost-2025-05-10-19-07-56-176  ]8;id=976952;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=619790;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\

--------!

In [23]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array

xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.array([float(x) for x in predictions.strip().split('\n')])
print(predictions_array.shape)

(12357,)


In [24]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [25]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))



Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



In [26]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[05/10/25 19:24:23] WARNING  The endpoint attribute has been renamed in sagemaker>=2.            ]8;id=462482;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=429908;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Deleting endpoint with name: sagemaker-xgboost-2025-05-10-19-07-56-176 ]8;id=571735;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=863385;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4985\4985]8;;\

[{'ResponseMetadata': {'RequestId': '9VWR1QTAXKRBP7G0',
   'HostId': '2ZaFeZC91Wwzw7PB+nT7aI9Ia31okZ/6aUIWT4eRTaYTr+/k54QTYFBWgrIlykHhorphvAg7r+ZEpZva68UIRXyFVsmTkVyW',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '2ZaFeZC91Wwzw7PB+nT7aI9Ia31okZ/6aUIWT4eRTaYTr+/k54QTYFBWgrIlykHhorphvAg7r+ZEpZva68UIRXyFVsmTkVyW',
    'x-amz-request-id': '9VWR1QTAXKRBP7G0',
    'date': 'Sat, 10 May 2025 19:24:24 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-05-10-18-44-01-587/debug-output/events/000000000030/000000000030_worker_0.tfevents'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-05-10-19-00-17-322/debug-output/events/000000000030/000000000030_worker_0.tfevents'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-05-10-19-00-17-322/profiler-output/frame